In [2]:
#Import Dependencies
!pip install tensorflow gpu opencv-python


ERROR: Could not find a version that satisfies the requirement gpu (from versions: none)
ERROR: No matching distribution found for gpu


In [3]:
!pip list

Package                           Version
--------------------------------- ---------------
absl-py                           2.1.0
aiobotocore                       2.5.0
aiofiles                          22.1.0
aiohttp                           3.8.5
aioitertools                      0.7.1
aiosignal                         1.2.0
aiosqlite                         0.18.0
alabaster                         0.7.12
altair                            5.2.0
anyio                             3.5.0
appdirs                           1.4.4
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.3
astroid                           2.14.2
astropy                           5.1
asttokens                         2.0.5
astunparse                        1.6.3
async-timeout                     4.0.2
atomicwrites                      1.4.0
attrs                             22.1.0
Automat                           20.2.0
autopep8              